In [1]:
!pip install transformers datasets accelerate peft bitsandbytes


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 1.9 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.6 MB/s eta 0:00:0000:0100:01


In [4]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="hf://datasets/toanduc/t5-sumary-dataset/dataset.csv")
dataset = dataset["train"].train_test_split(test_size=0.1)  

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
from transformers import AutoTokenizer

model_name = "VietAI/vit5-base"  
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = [doc for doc in examples["Original Text"]]
    targets = [doc for doc in examples["Summary"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    # print(labels)
    model_inputs["labels"] = labels["input_ids"]
    # print(model_inputs)
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/8948 [00:00<?, ? examples/s]

Map:   0%|          | 0/995 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

peft_config = LoraConfig(
    r=8,  # Rank của LoRA
    lora_alpha=32,  # Hệ số điều chỉnh
    lora_dropout=0.1,  # Dropout để tránh overfitting
    target_modules=["q", "v"],  # Chỉ fine-tune một số layer quan trọng
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # Kiểm tra số tham số được fine-tune


2025-03-09 08:05:11.455570: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 08:05:11.455607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 08:05:11.456364: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 08:05:11.460302: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

trainable params: 884,736 || all params: 226,835,712 || trainable%: 0.3900


In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    # per_device_train_batch_size=8,
    # per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    push_to_hub=False,
    report_to="none",
    per_device_train_batch_size=2, 
    per_device_eval_batch_size=2,  
    gradient_accumulation_steps=8,
    fp16=True, 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

trainer.train()


/home/jupyter-iec_iot13_toanlm/.local/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3755991/2186099578.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/tljh/user/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/opt/tljh/user/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was aske

Epoch,Training Loss,Validation Loss
0,No log,No log
1,8.664600,No log
2,8.664600,No log
3,2.222600,No log
4,2.222600,No log


/opt/tljh/user/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/tljh/user/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/opt/tljh/user/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but a

TrainOutput(global_step=1395, training_loss=4.50813246422771, metrics={'train_runtime': 12339.3986, 'train_samples_per_second': 3.626, 'train_steps_per_second': 0.113, 'total_flos': 2.730520769200128e+16, 'train_loss': 4.50813246422771, 'epoch': 4.9906124273580685})

In [14]:
import torch
def summarize(text):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
    model.to(device)  # Đảm bảo model cũng trên GPU

    output = model.generate(**inputs, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)

test_text = dataset["test"][0]["Original Text"]
print("Input:", test_text)
print("Summary:", summarize(test_text))


Input: Tuyển Việt Nam đứng im trên bảng xếp hạng FIFA
Dù đánh bại Palestine trong dịp FIFA Days vừa qua, nhưng tuyển Việt Nam vẫn đứng vị trí 95 trên bảng xếp hạng của Liên đoàn bóng đá thế giới.
Hai pha ghi bàn của Công Phượng và Phạm Tuấn giúp thầy trò HLV Philippe Troussier đánh bại các cầu thủ Palestine trong trận giao hữu trên sân Thiên Trường vừa qua.
Chiến thắng này giúp tuyển Việt Nam được công thêm 4,91 điểm để nâng tổng số điểm lên 1.243,14. Tuy nhiên, "Những chiến binh sao vàng" vẫn giữ nguyên thứ hạng 95 trên bảng xếp hạng FIFA và top 15 châu Á.
Điều này xuất phát từ việc các đội tuyển xếp trên thầy trò HLV Philippe Trousser cũng có được những kết quả tốt trong dịp FIFA Days vừa qua.
Trong khi đó, thất bại trước tuyển Việt Nam khiến Palestine bị trừ 8,97 điểm và tụt 1 bậc, từ vị trí 96 xuống 97.
Ở khu vực Đông Nam Á, ba đội tuyển gồm Thái Lan, Indonesia và Malaysia đều có sự thăng tiến so với tháng trước. Cụ thể, Thái Lan tăng từ 114 vươn lên 112 thế giới sau các trận đấu t

In [15]:
torch.save(model.state_dict(), "/home/jupyter-iec_iot13_toanlm/cv_match/model.pth")
print("Mô hình đã được lưu!")
# //load lại mô hình
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# model = AutoModelForSeq2SeqLM.from_pretrained("saved_model")
# tokenizer = AutoTokenizer.from_pretrained("saved_model")


Mô hình đã được lưu!


In [21]:
print("Summary:", summarize("tôi là người siêu đẹp trai"))

Summary: * là người mẫu đẹp trai, tôi là người mẫu đẹp trai. Tôi là người mẫu đẹp trai. Tôi là người mẫu đẹp trai. Tôi là người mẫu đẹp trai. Tôi là người mẫu đẹp trai. Tôi là người mẫu đẹp trai. Tôi là người mẫu đẹp trai. Tôi là người mẫu đẹp trai. Tôi là người mẫu đẹp trai. Tôi là người mẫu đẹp trai.
